# Convert df_events to meta dataframe for pyphotometry analysis
- Meant to be a bridge between the pyphotometry analysis and df_events for the time being 

In [11]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import seaborn as sns
import numpy as np
from trialexp.process.pycontrol.session_analysis import *
from trialexp.process.pycontrol.utils import *
from trialexp.process.pycontrol.session_analysis import Session


## Read task definition

In [12]:
basefolder = r'C:\code\trialexp'
tasks = pd.read_csv(basefolder+'\\params\\tasks_params.csv', usecols=[1, 2, 3, 4], index_col=False)
conditions, triggers, events_to_process = Session.get_task_specs(tasks, 'reaching_go_spout_incr_break2_nov22')

## Read pycontrol file

In [46]:

pycontrol_dir = 'Z:\\Teris\\ASAP\\pycontrol\\reaching_go_spout_nov22\\'
filename = 'kms062-2023-02-06-112451'
smrx_output_dir = 'Z:\\Teris\\ASAP\\pycontrol\\reaching_go_spout_nov22\\processed\\'

session = Session(pycontrol_dir+filename+'.txt')
df_events = parse_events(session)
# df_events = pd.read_pickle('df_events.pkl')
df_events[:60]

,state,event_name,time
1,short_break,state_change,0
2,short_break,bar_off,345
3,short_break,bar,425
4,short_break,bar_off,795
5,short_break,bar,1292
6,short_break,bar_off,1307
7,short_break,bar,1374
8,waiting_for_spout,state_change,1966
9,waiting_for_spout,CS_offset_timer,2000
10,waiting_for_spout,bar_off,2527


## Extract trial-related information from events

In [29]:
trial_window = [-2000, 4000]
timelim = [1000, 4000] # in ms

conditions, triggers, events_to_process = Session.get_task_specs(tasks, 'reaching_go_spout_nov22')

df_events_trials, df_events = extract_trial_by_trigger(df_events, triggers[0], conditions+events_to_process, 
                                            trial_window, 'test', session.datetime)
df_events_trials.head()

,US_end_timer_trial_time,bar_trial_time,bar_off_trial_time,spout_trial_time,trial_nb,uid,timestamp,trigger,valid
trial_nb,,,,,,,,,
1.0,"[19.0, 21.0]",[3964.0],[3881.0],[-1.0],1.0,test_2023-02-06_11:24:51_1.0,45572,busy_win,True
2.0,[20.0],"[-524.0, 663.0, 745.0, 1591.0, 1657.0]","[-597.0, -127.0, 709.0, 1576.0, 1594.0]","[-1.0, 26.0]",2.0,test_2023-02-06_11:24:51_2.0,61651,busy_win,True
3.0,[21.0],"[-649.0, -577.0, -511.0, 260.0, 333.0, 787.0, ...","[-718.0, -598.0, -561.0, -115.0, 318.0, 760.0,...",[0.0],3.0,test_2023-02-06_11:24:51_3.0,111702,busy_win,True
4.0,"[21.0, 1145.0]","[382.0, 1352.0, 1455.0, 1796.0]","[-66.0, 1063.0, 1439.0, 1683.0]","[0.0, 1124.0]",4.0,test_2023-02-06_11:24:51_4.0,120058,busy_win,True
5.0,"[21.0, 275.0, 365.0]","[350.0, 769.0]","[-100.0, 700.0]","[0.0, 254.0, 271.0, 292.0, 307.0, 327.0, 344.0...",5.0,test_2023-02-06_11:24:51_5.0,128554,busy_win,True


Compute conditions:

In [30]:
df_conditions = compute_conditions_by_trial(df_events_trials, conditions)
df_conditions.head()

KeyError: 'water_on_trial_time'

Compute success based on definition

In [28]:
session.create_metadata_dict(trial_window, timelim)

{'subject_ID': 62,
 'datetime': datetime.datetime(2023, 2, 6, 11, 24, 51),
 'timelim': [1000, 4000],
 'task': 'reaching_go_spout_nov22',
 'trial_window': [-2000, 4000],
 'com_port': 'COM3'}

In [ ]:
compute_success(df_events_trials, df_conditions, 